In [1]:
!pip install -q transformers accelerate bitsandbytes sentencepiece


In [8]:
from huggingface_hub import login
login()

In [10]:
!huggingface-cli whoami

nipungoel


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from collections import deque

# Configuration
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[*] Using device: {device}")

# Load model and tokenizer
print(f"[*] Loading model: {model_id}")
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True
)

chat = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Chat memory
history = deque(maxlen=5)  # Last 5 turns

def build_prompt(question):
    system_prompt = "[INST] You are a helpful, concise assistant. [/INST]"
    previous_turns = "\n".join(history)
    user_prompt = f"[INST] {question} [/INST]"
    return f"{system_prompt}\n{previous_turns}\n{user_prompt}".strip()

print("\n[*] Chatbot is ready! Type your questions below.")
print("Type '/exit' to end the chat.\n")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "/exit":
        print("Bot: Exiting chatbot. Goodbye! 👋")
        break

    prompt = build_prompt(user_input)

    output = chat(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        return_full_text=False
    )[0]["generated_text"].strip()

    print(f"Bot: {output}")
    history.append(f"[INST] {user_input} [/INST] {output}")


[*] Using device: cuda
[*] Loading model: mistralai/Mistral-7B-Instruct-v0.1


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0



[*] Chatbot is ready! Type your questions below.
Type '/exit' to end the chat.

You: hello


...

Bot: Hello! How can I assist you today?
You: What is the captial of france?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: The capital of France is Paris. Is there anything else I can help you with?
You: and india?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: The capital of India is New Delhi. Is there anything else I can help you with?
You: /exit
Bot: Exiting chatbot. Goodbye! 👋
